# 서울시 따릉이 이용정보 데이터

In [1]:
import pandas as pd 
import numpy as np

import warnings
warnings.filterwarnings('ignore')

데이터 로드

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv')

print(df.shape)
df.head()

(678950, 11)


,대여일자,대여시간,대여소번호,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,사용시간
0,2021-06-01,0,3541,정기권,F,~10대,1,0.00,0.00,0.00,8
1,2021-06-01,0,765,정기권,F,~10대,1,27.21,0.35,1526.81,19
2,2021-06-01,0,2637,정기권,F,~10대,1,41.40,0.37,1608.56,18
3,2021-06-01,0,2919,정기권,F,~10대,1,0.00,0.00,0.00,75
4,2021-06-01,0,549,정기권,F,~10대,1,13.04,0.17,731.55,6


31. 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜를 출력하라

In [3]:
ans1 = df['대여일자'].value_counts().sort_index().to_frame().reset_index()
ans1

,대여일자,count
0,2021-06-01,48081
1,2021-06-02,56100
2,2021-06-03,18985
3,2021-06-04,56788
4,2021-06-05,52730
5,2021-06-06,47787
6,2021-06-07,52623
7,2021-06-08,55623
8,2021-06-09,54507
9,2021-06-10,36992


In [4]:
ans2 = ans1[ans1['count'] == ans1['count'].max()].iloc[0, 0]
ans2

'2021-06-04'

32. 각 일자의 요일을 표기하고 ('Monday' ~'Sunday') 'day_name'컬럼을 추가하고 이를 이용하여 각 요일별 이용 횟수의 총합을 데이터 프레임으로 출력하라

In [5]:
df['대여일자'] = pd.to_datetime(df['대여일자'])
df['day_name'] = df['대여일자'].dt.day_name()

ans = df['day_name'].value_counts().to_frame().reset_index()
ans

,day_name,count
0,Wednesday,110607
1,Friday,108877
2,Monday,107568
3,Tuesday,103704
4,Saturday,101299
5,Sunday,90918
6,Thursday,55977


33. 각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력하라

In [6]:
ans = df.groupby(['day_name', '대여소번호'], as_index=False)['이용건수'].sum().sort_values(by=['day_name', '이용건수'], ascending=False)
ans = ans.drop_duplicates(subset='day_name', keep='first').reset_index(drop=True)
ans

,day_name,대여소번호,이용건수
0,Wednesday,502,399
1,Tuesday,502,371
2,Thursday,2715,188
3,Sunday,502,574
4,Saturday,502,616
5,Monday,2715,374
6,Friday,2715,414


34. 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대를 확인하라. 일일권의 경우 일일권 과 일일권(비회원)을 모두 포함하라

In [7]:
daily = df[df['대여구분코드'].str.contains('일일권')]['연령대코드'].value_counts().sort_index()
total = df['연령대코드'].value_counts().sort_index()

ratio = (daily / total).sort_values(ascending=False)
print(ratio)
print('-------------------------------------')

print('가장 높은 비율을 가지는 나이대 : ', ratio.index[0])

연령대코드
~10대    0.312525
20대     0.239913
30대     0.166702
40대     0.110585
50대     0.065414
70대~    0.048962
60대     0.042462
Name: count, dtype: float64
-------------------------------------
가장 높은 비율을 가지는 나이대 :  ~10대


35. 연령대별 평균 이동거리를 구하여라

In [8]:
ans = df.groupby('연령대코드', as_index=False)['이동거리'].mean()
ans

,연령대코드,이동거리
0,20대,3211.890552
1,30대,3341.443859
2,40대,3514.857416
3,50대,3593.668100
4,60대,3538.145737
5,70대~,3085.039641
6,~10대,2634.426279


36. 연령대 코드가 20대인 데이터를 추출하고,이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출한다.최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하여라

In [9]:
# 연령대 코드가 20대
twenty = df[df['연령대코드'] == '20대'].copy()

# 이동거리값이 이동거리값의 평균 이상인 데이터
twenty_mean = twenty[twenty['이동거리'] >= twenty['이동거리'].mean()]

# 대여일자, 대여소 번호 순서로 내림차순 정렬
ans = twenty_mean.sort_values(by=['대여일자', '대여소번호'], ascending=False)

# 1행부터 200행까지의 탄소량의 평균을 소수점 3째 자리까지 표현
ans = ans.iloc[:200]['탄소량'].astype('float').mean().round(3)
ans

1.613

37. 6월 7일 ~10대의 "이용건수"의 중앙값은?

In [10]:
ans = df[(df['대여일자'] == '2021-06-07') & (df['연령대코드'] == '~10대')]['이용건수'].median()
ans

1.0

38. 평일 (월~금) 출근 시간대(오전 6,7,8시)의 대여소별 이용 횟수를 구해서 데이터 프레임 형태로 표현한 후 각 대여시간별 이용 횟수의 상위 3개 대여소와 이용횟수를 출력하라

In [11]:
target = df[(~df['day_name'].isin(['Saturday', 'Sunday'])) & (df['대여시간'].isin([6, 7, 8]))]
target = target.groupby(['대여시간', '대여소번호'])['이용건수'].sum().to_frame('이용 횟수')

# 각 대여시간별 이용횟수의 상위 3개 대여소
ans = target.sort_values(by=['대여시간', '이용 횟수'], ascending=False).groupby('대여시간').head(3)
ans

이용 횟수
대여시간 대여소번호       
8    2701     360
     2715     155
     646      151
7    259      184
     2744     125
     2733     104
6    2744      55
     1028      47
     1125      44

39. 이동거리의 평균 이상의 이동거리 값을 가지는 데이터를 추출하여 추출데이터의 이동거리의 표본표준편차 값을 구하여라

In [12]:
ans = df[df['이동거리'] >= df['이동거리'].mean()]['이동거리'].std()
ans

5092.139707505305

40. 남성('M' or 'm')과 여성('F' or 'f')의 이동거리값의 평균값을 구하여라

In [13]:
df['sex'] = df['성별'].map(lambda x : '남' if x in ['M', 'm'] else '여')

ans = df.groupby('sex', as_index=False)['이동거리'].mean()
ans

,sex,이동거리
0,남,3209.110871
1,여,3468.575025


# 전세계 행복도 지표 데이터

데이터 로드

In [14]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv', encoding='utf-8')

print(df.shape)
df.head()

(312, 10)


,행복랭킹,나라명,점수,상대GDP,사회적지원,행복기대치,선택의 자유도,관대함,부패에 대한인식,년도
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,2019
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,2019
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,2019
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,2019
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,2019


41. 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다. 각년도의 행복랭킹 10위를 차지한 나라의 행복점수의 평균을 구하여라

In [15]:
ans = df[df['행복랭킹'] == 10]['점수'].mean()
ans

7.259

42. 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다. 각년도의 행복랭킹 50위이내의 나라들의 각각의 행복점수 평균을 데이터프레임으로 표시하라

In [16]:
df[df['행복랭킹'] <= 50].groupby('년도', as_index=False)['점수'].mean()

,년도,점수
0,2018,6.64678
1,2019,6.67002


43. 2018년도 데이터들만 추출하여 행복점수와 부패에 대한 인식에 대한 상관계수를 구하여라

In [17]:
target = df[df['년도'] == 2018].copy()
ans = target[['점수', '부패에 대한인식']].corr().iloc[0, 1]
ans

0.40529152271510027

44. 2018년도와 2019년도의 행복랭킹이 변화하지 않은 나라명의 수를 구하여라

In [18]:
ans = len(df[['행복랭킹', '나라명']]) - len(df[['행복랭킹', '나라명']].drop_duplicates())
ans

15

45. 2019년도 데이터들만 추출하여 각변수간 상관계수를 구하고 내림차순으로 정렬한 후 상위 5개를 데이터 프레임으로 출력하라. 컬럼명은 v1,v2,corr으로 표시하라

In [19]:
# unstack : 행렬을 시리즈 형태로 변환
corr = df[df['년도'] == 2019].select_dtypes(include=['int', 'float']).corr().unstack().reset_index().dropna()
corr.columns = ['v1', 'v2', 'corr']

corr = corr[corr['corr'] != 1].sort_values(by='corr', ascending=False).drop_duplicates(subset='corr')
ans = corr.head(5).reset_index(drop=True)
ans

,v1,v2,corr
0,행복기대치,상대GDP,0.835462
1,상대GDP,점수,0.793883
2,행복기대치,점수,0.779883
3,사회적지원,점수,0.777058
4,사회적지원,상대GDP,0.754906


46. 각 년도별 하위 행복점수의 하위 5개 국가의 평균 행복점수를 구하여라

In [20]:
ans = df.groupby('년도').tail(5).groupby('년도', as_index=False)['점수'].mean()
ans

,년도,점수
0,2018,3.1800
1,2019,3.1408


47. 2019년 데이터를 추출하고 해당 데이터의 상대 GDP 평균 이상의 나라들과 평균 이하의 나라들의 행복점수 평균을 각각 구하고 그 차이값을 출력하라

In [21]:
mean = df[df['년도'] == 2019]['상대GDP'].mean()

over = df[(df['년도'] == 2019) & (df['상대GDP'] >= mean)]['점수'].mean()
under = df[(df['년도'] == 2019) & (df['상대GDP'] < mean)]['점수'].mean()

ans = over - under
ans

1.390483957219251

48. 각년도의 부패에 대한인식을 내림차순 정렬했을때 상위 20개 국가의 부패에 대한인식의 평균을 구하여라

In [30]:
ans = df.sort_values(by=['년도', '부패에 대한인식'], ascending=False).groupby('년도').head(20).groupby('년도', as_index=False)['부패에 대한인식'].mean()
ans

,년도,부패에 대한인식
0,2018,0.3267
1,2019,0.3201


49. 2018년도 행복랭킹 50위 이내에 포함됐다가 2019년 50위 밖으로 밀려난 국가의 숫자를 구하여라

In [42]:
ans = set(df[(df['년도'] == 2018) & (df['행복랭킹'] <= 50)]['나라명']) - set(df[(df['년도'] == 2019) & (df['행복랭킹'] <= 50)]['나라명'])
ans = len(ans)
ans

4

50. 2018년,2019년 모두 기록이 있는 나라들 중 년도별 행복점수가 가장 증가한 나라와 그 증가 수치는?

In [74]:
# 2018년, 2019년 모두 기록이 있는 나라
count = df['나라명'].value_counts()
target = count[count >= 2].index

multiple = df[df['나라명'].isin(target)].reset_index(drop=True)

# 2018년도, 2019년도 데이터 필터링
df_2018 = multiple[multiple['년도'] == 2018][['나라명', '점수']].rename(columns={'점수' : '점수_2018'})
df_2019 = multiple[multiple['년도'] == 2019][['나라명', '점수']].rename(columns={'점수' : '점수_2019'})

# 데이터 병합
target = pd.merge(df_2018, df_2019, on='나라명', how='inner')

# 행복 점수 증가량
target['점수 증가량'] = target['점수_2019'] - target['점수_2018']

# 결과
country = target.sort_values(by='점수 증가량', ascending=False).iloc[0]['나라명']
num = target.sort_values(by='점수 증가량', ascending=False).iloc[0]['점수 증가량']

print("행복점수가 가장 많이 증가한 나라 : ", country)
print("행복점수 증가 수치 : ", num)

행복점수가 가장 많이 증가한 나라 :  Burundi
행복점수 증가 수치 :  0.8700000000000001


# 지역구 에너지 소비량 데이터

데이터 로드

In [75]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/consum/Tetuan%20City%20power%20consumption.csv')

print(df.shape)
df.head()

(52416, 9)


,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
0,1/1/2017 0:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,1/1/2017 0:10,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,1/1/2017 0:20,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


51. DateTime컬럼을 통해 각 월별로 몇개의 데이터가 있는지 데이터 프레임으로 구하여라

In [82]:
df['DateTime'] = pd.to_datetime(df['DateTime'])

ans = df.groupby(df['DateTime'].dt.month, as_index=False).size().sort_values(by='DateTime')
ans

,DateTime,size
0,1,4464
1,2,4032
2,3,4464
3,4,4320
4,5,4464
5,6,4320
6,7,4464
7,8,4464
8,9,4320
9,10,4464


52. 3월달의 각 시간대별 온도의 평균들 중 가장 낮은 시간대의 온도를 출력하라

In [88]:
target = df[df['DateTime'].dt.month == 3].copy()

ans = target.groupby(target['DateTime'].dt.hour)['Temperature'].mean().sort_values().values[0]
ans

11.506612903225806

53. 3월달의 각 시간대별 온도의 평균들 중 가장 높은 시간대의 온도를 출력하라

In [93]:
ans = target.groupby(target['DateTime'].dt.hour)['Temperature'].mean().sort_values(ascending=False).values[0]
ans

18.393602150537635

54. Zone 1 Power Consumption 컬럼의 value값의 크기가 Zone 2  Power Consumption 컬럼의 value값의 크기보다 큰 데이터들의 Humidity의 평균을 구하여라

In [95]:
target = df[df['Zone 1 Power Consumption'] > df['Zone 2  Power Consumption']].copy()

ans = target['Humidity'].mean()
ans

68.23624448055052

55. 각 zone의 에너지 소비량의 상관관계를 구해서 데이터 프레임으로 표기하라

In [100]:
ans = df.iloc[:, -3:].corr()
ans

,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
Zone 1 Power Consumption,1.000000,0.834519,0.750733
Zone 2 Power Consumption,0.834519,1.000000,0.570932
Zone 3 Power Consumption,0.750733,0.570932,1.000000


56. Temperature의 값이 10미만의 경우 A, 10이상 20미만의 경우 B,20이상 30미만의 경우 C, 그 외의 경우 D라고 할때 각 단계의 데이터 숫자를 구하여라

In [103]:
def temp(x) :
    if x < 10 :
        return 'A'
    elif x < 20 :
        return 'B'
    elif x < 30 :
        return 'C'
    else :
        return 'D'

df['temp'] = df['Temperature'].apply(temp)

ans = df['temp'].value_counts()
ans

temp
B    26993
C    21105
A     2874
D     1444
Name: count, dtype: int64

57. 6월 데이터중 12시의 Temperature의 표준편차를 구하여라

In [107]:
ans = df[(df['DateTime'].dt.month == 6) & (df['DateTime'].dt.hour == 12)]['Temperature'].std()
ans

2.049941782795103

58. 6월 데이터중 12시의 Temperature의 분산을 구하여라

In [108]:
ans = df[(df['DateTime'].dt.month == 6) & (df['DateTime'].dt.hour == 12)]['Temperature'].var()
ans

4.202261312849164

59. Temperature의 평균이상의 Temperature의 값을 가지는 데이터를 Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값은?

In [112]:
ans = df[df['Temperature'] >= df['Temperature'].mean()].sort_values(by='Temperature').iloc[3]['Humidity']
ans

87.9

60. Temperature의 중간값 이상의 Temperature의 값을 가지는 데이터를Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값은?

In [114]:
ans = df[df['Temperature'] >= df['Temperature'].median()].sort_values(by='Temperature').iloc[3]['Humidity']
ans

80.3